In [1]:
import sys, os
from pathlib import Path
if "../src" not in sys.path:
    sys.path.append("../src")
import dietcoke
from collections import Counter
from tqdm.auto import tqdm

In [2]:
authors = []
for fp in tqdm(dietcoke.fp_lst):
    f = open(fp, 'r', encoding='utf-8')
    corpus = f.readlines()
    for line in corpus:
        author = dietcoke.Text(line).author
        authors.append(author)

100%|██████████| 9/9 [00:30<00:00,  3.43s/it]


In [3]:
author_cnt = Counter(authors)
authors_uni = list(author_cnt.keys())

In [4]:
for author in authors_uni:
    if not author == '':
        author_split = author.split('、')
        author_last = author_split[-1]
        if len(author_last[:-1]) > 2:
            print(author_split, '->', author_last[-1])

['徐世昌輯'] -> 輯
['汪榮寶撰'] -> 撰
['Liang Qi Chao'] -> o
['孔安國傳'] -> 傳
['瞿曇悉達'] -> 達
['司馬承禎'] -> 禎
['司馬承禎撰'] -> 撰
['令狐德棻'] -> 棻
['原題司馬承禎述'] -> 述
['長孫無忌'] -> 忌
['杜光庭撰'] -> 撰
['釋玄奘奉敕譯'] -> 譯
['Du You'] -> u
['宇文懋昭'] -> 昭
['林希逸註'] -> 註
['耶律楚材'] -> 材
['劉處玄註'] -> 註
['張君房編'] -> 編
['劉基', '劉標', '劉宗[ding]'] -> ]
['太平老人'] -> 人
['原題 锺離權著呂喦傳'] -> 傳
['司馬光註'] -> 註
['Zhu Xi'] -> i
['完顏和素'] -> 素
['李白著，王琦輯註'] -> 註
['劉[Qing]'] -> ]
['天花藏主人'] -> 人
['[佚名]'] -> ]
['楊守敬撰'] -> 撰
['王闓運撰'] -> 撰
['横田惟孝'] -> 孝
['聖祖仁皇帝'] -> 帝
['亟齋居士'] -> 士
['汪孟[Xuan]'] -> ]
['乾隆十三年'] -> 年
['易順鼎撰'] -> 撰
['王國維撰'] -> 撰
['服部誠一'] -> 一
['順治世祖章皇帝'] -> 帝
['吳慶坻撰'] -> 撰
['汪繼培箋'] -> 箋
['世宗憲皇帝'] -> 帝
['嗤嗤道人'] -> 人
['陸繼輅', '陸錢惠尊'] -> 尊
['翁洲老民'] -> 民
['柴野栗山'] -> 山
['長岡春一'] -> 一
['康熙聖祖仁皇帝'] -> 帝
['丘逢甲撰'] -> 撰
['康有為撰'] -> 撰
['訥齋道人'] -> 人
['吉田良太郎', '八詠樓主人'] -> 人
['愛新覺羅永瑆'] -> 瑆
['雍正世宗憲皇帝'] -> 帝
['乾隆十二年'] -> 年
['空空主人'] -> 人
['尾藤二洲'] -> 洲
['酌元亭主人'] -> 人
['九容樓主人'] -> 人
['嚴虞惇', '平陽主人'] -> 人
['隨緣下士編'] -> 編
['章炳麟撰'] -> 撰
['小藤文次郎'] -> 郎
['Sun Yi-Rang'] -> g
[